# Background Information

## Introduction 

Since the advent of quantitative photometric studies of galaxies, measuring their true sizes and fluxes has proven to be a challenging task. This is because the true extent of a galaxy can be hard to define: it can vary with morphology, be difficult to distinguish from its neighbors, or difficult to measure due to low surface brightness features. Over time, researchers have proposed various approaches for estimating the fluxes, shapes, and angular (or projected) sizes of galaxies. A powerful technique for measuring accurate properties of galaxies is to combine parametric measurements with an analysis of the profile of the light distribution of a galaxy.  

Parametric descriptions of galaxies were first introduced by  de Vaucouleurs (1948), who proposed a power-law profile that reasonably models the projected intensity profiles of galaxies. He used an $exp \{-k r^{1/4}\}$ intensity profile to describe the radial light distribution of elliptical galaxies. Following this model's success, Sérsic (1963, 1968) introduced a generalized version that can be applied to galaxies of various morphologies.

Though the sizes of galaxies can be measured by fitting a Sérsic model to an image, various radii of interest can also be derived from radial photometric measurements.  Petrosian (1976) formulated a radial profile, now named after him, as a means of defining the projected radii of galaxies. Since the Petrosian profile is a ratio of surface brightnesses, it offers a distance-independent method of measuring galaxy radii. Petrosian radii are useful for computing radial concentrations of galaxy light profiles and for performing accurate measurements of  galaxy photometry. Petrosian radii and concentrations can also be used to estimate the parameters of Sérsic profiles. 

By combining parametric fitting and radial profile analysis, accurate fluxes, shapes, and sizes can be derived for studying the properties and evolution of galaxies. PetroFit, a Python package based on the Astropy and Photutils packages, was motivated by the need for an open-source package to compute Petrosian radii and magnitudes. The prime motivation for constructing this package was to make a robust and scalable software package for modeling Sérsic and Petrosian profiles. PetroFit is designed to work within the Astropy ecosystem with a focus on measuring accurate galaxy properties.  Following Crawford (2006), PetroFit uses correction grids to improve the size estimates produced by Petrosian profiles. PetroFit also provides a robust Python-based parameter fitter code based on Astropy models for parameterization of galaxy properties. In this section, we will be defining morphological properties that can be measured using the PetroFit package.

## Sérsic Profiles

The Sérsic profile is a mathematical function that describes how the intensity of a galaxy varies with radius `r` from its center. The profile can be described using the following formulation:

<center>$I\left( r(x,y) \right) = I_e\exp\left\{-b_n\left[\left(\frac{r}{r_{e}}\right)^{(1/n)}-1\right]\right\}$</center>

The total flux within a radius `r` is giving by the following integral:

<center>$L(\leq r) = 2 \pi \int_{0}^{r} I(r^\prime) r^\prime dr^\prime$</center>

Where:

- $I$ is the intensity or flux at position `(x, y)`.
- $r$ is the radius (angular) from the center that corresponds to `(x, y)`.
- $r_{e}$ is the effective radius which is equal to the half-light radius.
- $I_e$ is the flux at the half light radius or $I_e = I(r_{e})$. 
- $n$ is the Sérsic index which determines the “steepness” of the profile. `n = 4` corresponds to a de Vaucouleurs' profile while `n = 1` gives the exponential profile.

The constant $b_n$ is defined such that $r_e$ contains half the total flux, and can be solved for numerically using:

$\Gamma(2n) = 2\gamma (2n,b_n)$

In `python`, the Sérsic $b_n$ can be calculated as follows:

In [ ]:
from scipy.special import gammaincinv
n = 4 # Sersic index 
b_n = gammaincinv(2. * n, 0.5)

###  Ellipticity and Elongation

Galaxies that can be described with a Sérsic profile are usually not perfectly symmetrical and often display an elliptical distribution. We define two quantities that can be used to describe the elliptical components of galaxies. Ellipticity and elongation are defined as follows:

<center>$ellip =  1  -  \frac{b}{a} = \frac{elong - 1}{elong}$</center>

<center>$elong = \frac{a}{b} = \frac{1}{1 - ellip}$</center>

Where $a$ is the unit length of the semi-major axis and $b$ is the corresponding semi-minor axis. A circle corresponds to $ellip = 0$ and $elong = 1$. Ellipticity ranges from 0 to 1 while elongation ranges from 1 to infinity. The SExtractor and  [Photutils](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.SourceCatalog.html\#photutils.segmentation.SourceProperties.elongation) packages use elongations for apertures while the Astropy-modeling sub-module uses ellipticity for [Sérsic models](https://docs.astropy.org/en/v4.2/api/astropy.modeling.functional_models.Sersic2D.html).  

### Input Raidus 

Elliptical Sérsic profiles can be described using $ellip$ (ellipticity) and $\theta$ (rotation angle). In such cases, the radius is given by the following expression:

<center>$r(x, y, \theta, ellip) = \sqrt{\left(r_{maj}(x, y)\right)^2 - \left(\frac{r_{min}(x, y)}{(1 - ellip)}\right)^2}$</center>

Where:

$r_{maj}(x, y) = (x - x_0)\cdot{cos(\theta)} + {(y - y_0)}\cdot sin(\theta)$

$r_{min}(x, y) = -(x - x_0)\cdot{sin(\theta)} + {(y - y_0)}\cdot cos(\theta)$


### Sérsic Index

A Sérsic index determines the degree of curvature of the profile. The larger values of $n$ get, the more concentrated the profile is at smaller radii. As $n$ gets larger, the Sérsic model tends to a power-law with a slope equal to 5. `n = 4` corresponds to a de Vaucouleurs' profile which describes elliptical galaxies and the cores of spirals well; while `n = 1` gives the exponential profile, which models spiral disks well. `n = 0.5` gives a low concentration profile (Gaussian) that can be used to model inter-cluster glow in galaxy clusters (assuming an elliptical shape), and can be used as an alternative to a  Ferrer  Profile.   


![Sersic index align="center"](images/sersic_index.png)

### Effective Radius and Intensity

$r_{e}$ is the effective radius or half-light radius which encloses half of the total flux of the galaxy and can be defined by the following expression:

$L(\leq r_e) = \frac{1}{2}L(\leq \infty)$. 

$I_e$ is the intensity or flux exactly at $r_{e}$ and can be defined as $I_e = I(r_{e})$ and determines the amplitude of the profile. $I_e$ can be related to the intensity at the center of the Sérsic profile ($I_0 = I(r_0)$) as follows:

$I_e = \frac{I_0}{\exp\left\{b_n\right\}}$



![Sersic Radii align="center"](images/sersic_radii.png)

In this plot we see a one dimensional cross-section of a  Sérsic Profile (arbitrary units) with $r_e = 25$ and $I_e = 1$. Notice how $I_0$ is equal to 5.3 and not 1. The dotted blue area under the curve contains half of the total flux and is equal to the striped green area in value (the green area extends to infinity).

### Curve of Growth

The curve of growth (COG) of a galaxy is the relationship between radius (from the galaxy center) and the total flux within that radius. It represents the cumulative flux enclosed at a given radius. The COG is very sensitive to background noise and fluctuations from the galaxy's stochastic components. For a galaxy well described by a Sersic profile, the curve of growth is given by $L(\leq r)$.

Below is a plot of the COG of a Sérsic profile with $(I_e=1, r_e=25,  n=1)$:

![cog plot align="center"](images/sersic_cog.png)

## Petrosian Profiles 

In `petrofit`, the Petrosian profile or Petrosian index $\eta$ is defined as the inverse of the original formulation by Petrosian (1976), and has the property where $\eta(0) = 1$ at the center of the galaxy and drops to zero at the edge:

<center>$\eta (r) = \frac{I(r)}{\langle I(r) \rangle} = I(r) \frac{A(r)}{L(\leq r)}$</center>

Where:

- $\eta (r)$ is the Petrosian at $r$.
- $I(r)$ is the surface brightness at $r$.
- $\langle I (r) \rangle$ is the average surface brightness within $r$.
- $L(\leq r)$ is the total flux within $r$.
- $A(r)$ is the aperture area. For elliptical apertures: $A(r) = \pi \cdot (1 - ellip) \cdot r^2$  

 

 

Below is a plot Petrosian profile of a Sérsic model with $(I_e=1, r_e=25,  n=1)$:

![sersic_petrosian align="center"](images/sersic_petrosian.png)

### Discrete Computation

Given an array of enclosed fluxes ($L$) and corresponding aperture areas ($A$), the Petrosian profile can be computed discretely as follows:

1. Estimate the surface brightness by finding the average flux between the current index ($i$) and the last index ($i-1$). Note that the gap between apertures affects the accuracy of the surface brightness at $i$, thus it is recommended to use apertures with radii that are incremented by a small number of pixels:

\begin{equation}
I[i] \approx \frac {L[i] - L[i-1]} {A[i] - A[i-1]}
\end{equation}

2. Estimate the average surface brightness by taking the flux at $i$ and dividing it by the corresponding aperture area:

\begin{equation}
{\langle I[i] \rangle} = \frac {L[i]} {A[i]}
\end{equation}

3. Compute the Petrosian index at $i$ using the estimated values in steps 1 and 2:

\begin{equation}
\eta [i] = \frac{I[i]}{\langle I[i] \rangle} = \left(\frac {L[i] - L[i-1]} {A[i] - A[i-1]}\right) \cdot \frac {A[i]} {L[i]}
\end{equation}

In discrete computations, the Petrosian profile can not be computed at the first index even if it corresponds to the center pixel ($r[i_0]=0$).  In real images, the surface brightness of a galaxy is binned into pixels, and to accurately determine $I_0$, one would need to infinitely oversample the central region. In other words, each pixel corresponds to a total surface brightness integrated within the area of the pixel as opposed to the surface brightness at the pixel coordinates. As such, PetroFit sets the first Petrosian value to \texttt{numpy.nan} when returning a discretely computed array of Petrosian indices. PetroFit takes advantage of fact that the Petrosian index at the center of a galaxy is equal to $1$ when computing radii internally.

### Petrosian Radius and Eta 

The Petrosian radius is the radius at which the Petrosian profile is equal to a special Petrosian index $\eta_{petro}$. In `petrofit`, the Petrosian radius is defined as the radius where $\eta = 0.2$. Thus the Petrosian radius and $\eta_{petro}$ ("eta") can be defined as:

<center>$\eta( r_{petro} ) = \eta_{petro} = 0.2$</center>


Here is a plot of the Petrosian radius and $\eta=0.2$ for a Sérsic  profile with $(I_e=1, r_e=25,  n=1)$:

![sersic_petrosian_eta align="center"](images/sersic_petrosian_eta.png)


### Petrosian Total Flux Radius and Epsilon

The Petrosian total flux radius is, as the name implies, the radius which encloses the total flux of the galaxy. Since Sérsic profiles extend to infinity, it is not possible to define a radius that contains 100% of the total flux (that would equate to a radius at infinity). It is also not practical to consider minute flux values at very large radii because real images of galaxies contain noise. For these reasons `petrofit` considers the radius that contains 99% of the galaxy’s light to be the total flux radius. To attain this radius, we must multiply the Petrosian radius with a constant. This constant is called epsilon and is set to 2 by default. Thus we can define the Petrosian total flux radius as follows:

<center>$r_{total} = \epsilon \cdot r_{petro}$</center>

### Petrosian Half Light Radius

The Petrosian half-light radius is the radius which contains half of the total flux. This quaintly is especially important because the half-light radius is equal to the Sérsic effective radius. The half-light radius can be computed by numerically finding the radius that encloses half of the total flux. Consequently, the Petrosian half-light radius can be defined as the radius which satisfies the following expression:

<center>$L(\leq r_{hl}) = \frac{1}{2} L(\leq r_{total} )$</center>

### Concentration Index

The concentration index is the ratio of the radii containing a specified fraction of the total flux and can be generalized as follows:  

\begin{equation}
C_{i o} \equiv 5 \cdot log\left(\frac{r_{o}}{r_{i}}\right)
\end{equation}

where $r_o$ and $r_i$ are the outer and inner radii, enclosing $o\%$ and $i\%$ percent of the total flux respectively. The concentration index is related to the profile shape and can be used as a proxy for morphology. Some commonly used concentration indices are $C_{2080}$ and $C_{5090}$:

\begin{equation}
C_{2080} = 5 \cdot log\left(\frac{r_{80}}{r_{20}}\right)
\end{equation}

\begin{equation}
C_{5090} = 5 \cdot log\left(\frac{r_{90}}{r_{50}}\right)
\end{equation}

$C_{2080}$ correlates better with the Sérsic index (and epsilon). However, in the presence of image degradation, $C_{5090}$ may be more immune from systematic effects.

### Petrosian Corrections 

To first order, the Petrosian magnitude is independent of redshift as it depends on the ratio of two surface brightnesses, but because it is based on profile shape, changes in the profile due to morphological K-corrections still apply. However, the real strength of Petrosian magnitudes vs. other variable aperture photometry such as Kron magnitudes (Kron 1980) is that the Petrosian radius only depends on light interior to this radius. This aspect leads to small random errors in the measurement of $r_{petro}$. Nonetheless, the magnitude within $2 \cdot r_{petro}$, relative to a true, total magnitude, is still profile dependent. Although 99% of the light from an exponential profile ($n=1$) is recovered within $2 \cdot r_{petro}$, only 83.4% of the light for an $r^{1/4}$ profile is measured within $2 \cdot r_{petro}$ (Graham et al. 2005).

Because of this, we need to adjust $r_{total}$ by finding the appropriate epsilon ($\epsilon$) value. Both the consternation index and "correct" $r_{total}$ are related, thus a relationship between the consternation index and $\epsilon$ can be derived by simulating Sérsic profiles. For ideal images of Sérsic models (no noise and no PSF), we can make a plot of $C2080$ vs Epsilon of simulated profiles with different half-light radii:

![concentration_index_vs_epsilon align="center"](images/concentration_index_vs_epsilon.png)

The scatter that is seen in higher concentration is due to sampling of the model onto a pixel grid. Profiles with small half-light radii and high concentrations are especially affected because they require a large oversampling factor.

Real images of galaxies contain noise and light from a source is smeared according to the PSF of the instrument. The smearing of light results in the reduction of concentration, causing a skew in the relationship we derived above. To illustrate this, the plot below shows concentration indices for the same profiles as above, but convolved with a Hubble Space Telescope `F105W` PSF, sampled at `60 mas`. It also shows the same polynomial fit (red line) as above for reference:

![f105w_concentration_index_vs_epsilon align="center"](images/f105w_concentration_index_vs_epsilon.png)

Notice how profiles with smaller half-light radii are affected (skewed) the most by the PSF convolution. For this reason, we must make corrections that account for the half-light radius and the shape of the PSF as well. 

## List of Galaxy Properties

The following galaxy properties can be computed using `petrofit` as described in this section:


**Sersic Parameters**

- [**amplitude**: Surface brightness at r_eff.](./fitting.ipynb#Amplitude-at-r_eff)
- [**r_eff**: Effective (half-light) radius.](./fitting.ipynb#Half-Light-Radius-(r_eff))
- [**n**: Sersic Index.](./fitting.ipynb#Sérsic-Index)
- [**x_0 and y_0**: x and y position of the center.](./fitting.ipynb#Center-Pixel)
- [**ellip**:  Ellipticity of the profile.](./fitting.ipynb#Ellipticity-and-Elongation)
- [**elong**:  Elongation of the profile.](./fitting.ipynb#Ellipticity-and-Elongation)
- [**theta**: Rotation angle.](./fitting.ipynb#Source-Orientation)

**Petrosian Parameters**

- [**r_petrosian**: Petrosian Radius.](./photometry_and_petrosian.ipynb#Petrosian-Radius)
- [**r_total_flux**: Petrosian Total Flux Radius.](./photometry_and_petrosian.ipynb#Petrosian-Total-Flux-Radius)
- [**r_half_light**: Petrosian Half Light Radius.](./photometry_and_petrosian.ipynb#Petrosian-Half-Light-Radius)
- [**total_flux**: Total Petrosian Flux](./photometry_and_petrosian.ipynb#Total-Petrosian-Flux)
- [**concentration_index**: Concentration Index.](./photometry_and_petrosian.ipynb#Concentration-Index)
- [**epsilon**: Estimated Corrected Epsilon.](./photometry_and_petrosian.ipynb#Estimate-corrected-epsilon)

## References 


Bershady, M. A., Jangren, A., and Conselice, C. J., “Structural and Photometric Classification of Galaxies. I. Calibration Based on a Nearby Galaxy Sample”, <i>The Astronomical Journal</i>, vol. 119, no. 6, pp. 2645–2663, 2000. doi:10.1086/301386.

Crawford, S. M., “The distribution of star-forming galaxies in intermediate redshift galaxy clusters”, PhDT, 2006.

de Vaucouleurs, G. 1948, Annales d’Astrophysique, 11, 247

Graham, A. W., “Total Galaxy Magnitudes and Effective Radii from Petrosian Magnitudes and Radii”, <i>The Astronomical Journal</i>, vol. 130, no. 4, pp. 1535–1544, 2005. doi:10.1086/444475.

Graham, A. W. and Driver, S. P., “A Concise Reference to (Projected) Sérsic R<SUP>1/n</SUP> Quantities, Including Concentration, Profile Slopes, Petrosian Indices, and Kron Magnitudes”, <i>Publications of the Astronomical Society of Australia</i>, vol. 22, no. 2, pp. 118–127, 2005. doi:10.1071/AS05001.

Kent, S. M., “CCD Surface photometry of field galaxies.I. Observations.”, <i>The Astrophysical Journal Supplement Series</i>, vol. 56, pp. 105–141, 1984. doi:10.1086/190978.

Kron, R. G., “Photometry of a complete sample of faint galaxies.”, <i>The Astrophysical Journal Supplement Series</i>, vol. 43, pp. 305–325, 1980. doi:10.1086/190669.

Petrosian, V., “Surface Brightness and Evolution of Galaxies”, <i>The Astrophysical Journal</i>, vol. 210, p. L53, 1976. doi:10.1086/182301.

Sérsic, J. L. 1963, Boletin de la Asociacion Argentina de
Astronomia La Plata Argentina, 6, 41
—. 1968, Atlas de Galaxias Australes

